<a href="https://www.kaggle.com/code/tunguz/mnist-xgboost-baseline?scriptVersionId=96778853" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
col = ['pixel%d'%i for i in range(784)]

In [3]:
target = train[['label']].values
train.drop(['label'], axis=1, inplace=True)

In [4]:
features = train.columns

In [5]:
%%time
train_test = pd.concat([train, test], axis =0)
RS = RobustScaler()
RS.fit(train_test)
del train_test
gc.collect()
gc.collect()
train[features] = RS.transform(train)
test[features] = RS.transform(test)
del RS
gc.collect()
gc.collect()

CPU times: user 42.5 s, sys: 1min 23s, total: 2min 6s
Wall time: 2min 6s


0

In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(train, target, test_size=0.1, random_state=42)
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

In [7]:
%%time

watchlist = [(dval, 'eval'), (dtrain, 'train')]

xgb_params = {'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.0014345301923969806,
 'alpha': 0.4687380486697353,
 'colsample_bytree': 0.8434345456331338,
 'subsample': 0.8991048362147761,
 'learning_rate': 0.01,
 'max_depth': 21,
 'min_child_weight': 4,
 'num_class': 10,
 'eval_metric': 'merror'}

xgb_clf = xgb.train(xgb_params, dtrain, 20000, 
                    watchlist, 
                    #early_stopping_rounds=40, 
                    verbose_eval=100)

[0]	eval-merror:0.109762	train-merror:0.060847
[100]	eval-merror:0.040714	train-merror:0.007989
[200]	eval-merror:0.039048	train-merror:0.003466
[300]	eval-merror:0.037619	train-merror:0.00172
[400]	eval-merror:0.036429	train-merror:0.000926
[500]	eval-merror:0.034762	train-merror:0.000317
[600]	eval-merror:0.034048	train-merror:0.000185
[700]	eval-merror:0.032143	train-merror:0.000106
[800]	eval-merror:0.030952	train-merror:5.3e-05
[900]	eval-merror:0.030714	train-merror:2.6e-05
[1000]	eval-merror:0.030238	train-merror:2.6e-05
[1100]	eval-merror:0.03	train-merror:2.6e-05
[1200]	eval-merror:0.029762	train-merror:0
[1300]	eval-merror:0.028571	train-merror:0
[1400]	eval-merror:0.02881	train-merror:0
[1500]	eval-merror:0.02881	train-merror:0
[1600]	eval-merror:0.028095	train-merror:0
[1700]	eval-merror:0.027381	train-merror:0
[1800]	eval-merror:0.027381	train-merror:0
[1900]	eval-merror:0.027143	train-merror:0
[2000]	eval-merror:0.027143	train-merror:0
[2100]	eval-merror:0.027143	train-me

In [8]:
res_prob = xgb_clf.predict(xgb.DMatrix(test[col]))


In [9]:
res_prob

array([[5.3165713e-06, 1.4337697e-07, 9.9998581e-01, ..., 1.3239763e-06,
        1.8112149e-06, 1.2163831e-06],
       [9.9996996e-01, 1.4588447e-06, 8.4198628e-06, ..., 2.2628321e-06,
        7.1202709e-07, 2.5506486e-07],
       [1.1493453e-05, 2.7996413e-05, 2.5294066e-04, ..., 1.3576147e-04,
        4.2694851e-04, 9.9819690e-01],
       ...,
       [1.7732376e-07, 4.9730585e-07, 2.4183205e-07, ..., 1.9171861e-07,
        2.0714142e-06, 1.2908238e-06],
       [2.0932519e-05, 9.0667112e-07, 3.8167386e-07, ..., 9.6563945e-06,
        2.3657108e-06, 9.9958187e-01],
       [1.7082168e-06, 2.5690525e-07, 9.9996877e-01, ..., 8.9319417e-07,
        4.9083351e-06, 1.1594834e-05]], dtype=float32)

In [10]:
np.argmax(res_prob, axis=1)

array([2, 0, 9, ..., 3, 9, 2])

In [11]:
val_preds = np.argmax(xgb_clf.predict(dval), axis=1)

In [12]:
val_preds

array([8, 1, 9, ..., 7, 7, 5])

In [13]:
accuracy_score(Y_val, val_preds)

0.9745238095238096

In [14]:
res = np.argmax(res_prob, axis=1)

In [15]:
res.astype('int')

array([2, 0, 9, ..., 3, 9, 2])

In [16]:
sample_submission['Label'] = res.astype('int')
sample_submission.to_csv('submission.csv', index=False)
np.save('res_prob', res_prob)